In [3]:
import importlib

import wrds
import pandas as pd
import datetime
import numpy as np
import pickle
from time import time, sleep
import argparse as ap
import sys
import csv
from dateutil.relativedelta import relativedelta

db = wrds.Connection()

AttributeError: module 'wrds' has no attribute 'Connection'

In [51]:
def option_table(date):
    s = 'opprcd' + date[0:4]
    return s

def adj_date(secid, date):
    q1 = ("select secid, date, exdate "
      "from optionm.%s "
      "where cp_flag = 'P' and secid = %g and date >= '%s'" % (option_table(date), secid, date) )
    df = db.raw_sql(q1)
    return df.date.unique()[0].strftime('%Y-%m-%d')

def get_secid(ticker, date):
    q1 = ("select secid, optionid, ticker, effect_date "
      "from optionm.optionmnames "
      "where ticker = '%s' and effect_date < '%s'" % (ticker, date) )
    df = db.raw_sql(q1)
    return df.sort_values(by=['effect_date']).iloc[-1].secid

def target_ex_date(start_date, days):
    date_1 = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end_date = date_1 + datetime.timedelta(days=days)
    #date_string = end_date.strftime('%Y-%m-%d')
    return end_date.date()

def available_ex_dates(secid,date):
    q1 = ("select secid, date, exdate "
      "from optionm.%s "
      "where cp_flag = 'P' and secid = %g and date = '%s'" % (option_table(date), secid, date) )
    df = db.raw_sql(q1)
    return df.exdate.unique()

def available_strikes(secid,date,ex_date):
    q1 = ("select secid, date, exdate, strike_price "
      "from optionm.%s "
      "where cp_flag = 'P' and secid = %g and date = '%s' and exdate = '%s'" % (option_table(date), secid, date, ex_date) )
    df = db.raw_sql(q1) 
    strikes = df.strike_price.unique()
    return [x / 1000 for x in strikes]

def nearest(list, pivot):
    return min(list, key=lambda x: abs(x - pivot))

def one_day_pricing(secid,date,ex_date):
    q1 = ("select secid, date, symbol, optionid, exdate, strike_price, best_bid, best_offer "
      "from optionm.%s "
      "where cp_flag = 'P' and secid = %g and date = '%s' and exdate = '%s'" % (option_table(date), secid, date, ex_date) )
    df = db.raw_sql(q1)
    return df

def one_contract_pricing(secid,date,ex_date,strike_price):
    q1 = ("select secid, date, symbol, optionid, exdate, strike_price, best_bid, best_offer "
      "from optionm.%s "
      "where cp_flag = 'P' and secid = %g and date >= '%s' and exdate = '%s' and strike_price = %g" % (option_table(date), secid, date, ex_date, strike_price) )
    df = db.raw_sql(q1)
    return df

def hi_lo_avg(secid,date):
    q1 = ("select secid, date, high, low "
      "from optionm.secprd "
      "where secid = %g and date = '%s'" % (secid, date) )
    df = db.raw_sql(q1)
    return round((df.high + df.low)/2,2)

def nearest_strike(secid,date,delta,strikes):
    p = hi_lo_avg(secid, date)
    p_target = np.float64(p * delta)
    s = nearest(strikes,p_target)
    if s > p * (delta + (1-delta)/2):
        p_target = np.float64(p * (delta - (1-delta)/2))
        s = nearest(strikes,p_target)
    return s * 1000

def nearest_end_date(end_dates, end_date_date):
    date2 = end_date_date
    while True:
        date = nearest(end_dates,date2)
        if min(end_dates) > end_date_date:
            date = date2
            break
        if date <= end_date_date:
            break
        date2 = date2 - datetime.timedelta(days=1)
    return date

def average_ex_date(results):
    d=[]
    for t in results:
        d.append(results[t]['expiry']) 
    a = np.array(list(d), dtype='datetime64').view('i8').mean().astype('datetime64[D]')
    return pd.to_datetime(str(a))

def cost_final(ticker,start_date,end_date,delta,fwd,verbose=False):
    sec_id = get_secid(ticker, start_date)
    if verbose:
        print("Security ID: " + str(sec_id))
    start_date = adj_date(sec_id,start_date)
    if verbose:
        print("Adjusted Start Date: " + start_date)
    dates = available_ex_dates(sec_id, start_date)
    if verbose:
        print("Available ex dates: ")
        print(dates)
    target_ex = target_ex_date(start_date,fwd)
    if verbose:
        print("Target ex date: " + target_ex.strftime('%Y-%m-%d'))
    nearest_ex = nearest(dates,target_ex)
    if verbose:
        print("Nearest ex date: " + nearest_ex.strftime('%Y-%m-%d'))
    strikes = available_strikes(sec_id, start_date,nearest_ex)
    if verbose:
        print("Available strikes: ")
        print(strikes)
    hl = hi_lo_avg(sec_id,start_date)
    if verbose:
        print("Current price: " + str(hl)) 
        print("Target strike: " + str(round((hl * delta),2)))  
    strike = nearest_strike(sec_id,start_date,delta,strikes)
    if verbose:
        print("Closest strike: " + str(strike/1000))
        print('.............................')
        print('.............................')
    prices = one_contract_pricing(sec_id,start_date,nearest_ex,strike)
    start_bid = prices.iloc[0].best_bid
    start_ask = prices.iloc[0].best_offer
    cost = round((start_bid * 0.67) + (start_ask * 0.33),3)
    cost_date = prices.iloc[0].date
    if end_date is None:
        final_bid = prices.iloc[-1].best_bid
        final_ask = prices.iloc[-1].best_offer
        final = round((final_bid * 0.33) + (final_ask * 0.67),3)
        final_date = nearest_ex.strftime('%Y-%m-%d')
    else:
        end_dates = prices.date.unique()
        end_date_date = datetime.datetime.strptime(end_date, "%Y-%m-%d").date()
        closest_end_date = nearest_end_date(end_dates,end_date_date)
        prices_data = prices.loc[prices['date'] == closest_end_date]
        final_bid = prices_data.best_bid.iloc[0]
        final_ask = prices_data.best_offer.iloc[0]
        final = round((final_bid * 0.33) + (final_ask * 0.67),3)
        final_date = prices_data.date.iloc[0]
    s_yield = round(cost / (strike/1000*1.00),4)
    hp = (nearest_ex - cost_date).days*1.00/360.00
    a_yield = round(((1.00 + s_yield)**(1.00/hp)) - 1.00,4)
    d= {
            "ticker" : ticker,
            "trade_date" : cost_date.strftime('%Y-%m-%d'),
            "expiry" : nearest_ex.strftime('%Y-%m-%d'),
            "current" : hl,
            "strike" : strike/1000,
            "delta" : round(strike/1000/hl - 1,2),
            "c_date" : cost_date.strftime('%Y-%m-%d'),
            "c_bid" : start_bid,
            "c_ask" : start_ask,
            "c_mid" : cost,
            "f_date" : str(final_date),
            "f_bid" : final_bid,
            "f_ask" : final_ask,
            "f_mid" : final,
            "gross_profit" : (cost - final),
            "s_yield" : s_yield,
            "a_yield" : a_yield
            }
    return d

def calculate_number_of_contracts(starting_capital,results,max_pos_size):
    number_of_contracts = len(results.values())
    target_dollar_per_position = starting_capital / number_of_contracts
    for r in results.values():
        contracts = round(target_dollar_per_position/r["strike"]/100,0)
        if contracts < 1:
            r["contracts"] = 1
        else:
            r["contracts"] = contracts
        if target_dollar_per_position > (starting_capital * max_pos_size):  
            r["target_notional"] = (starting_capital * max_pos_size)
        else:
            r["target_notional"] = round(target_dollar_per_position,2)
        r["actual_notional"] = round(contracts * r["strike"] * 100, 2)
        r["total_profit"] = round(contracts * r["gross_profit"] * 100,2)
        r["ending_total"] = r["target_notional"] + r["total_profit"] 
    return results

def calculate_annualized_yield(results):
    for r in results.values():
        r["s_yield"] = round(r["c_mid"] / r["strike"],2)
        hp = (datetime.datetime.strptime(r["expiry"], "%Y-%m-%d") - datetime.datetime.strptime(r["trade_date"], "%Y-%m-%d"))/365
        r["a_yield"] = ((1 + r["s_yield"])**(1/hp)) - 1
    return results

def update_starting_capital(results):
    total = 0 
    for r in results.values():
        total = total + r["ending_total"]
    return total

def update_total_pl(start_date,total_pl):
    total = 0 
    for r in results.values():
        total = total + r["ending_total"]
    return {'end_date' : start_date,'market_value' : total}

def pretty_print(cf_data):
    print("Trade date:" + cf_data["trade_date"])
    print("Expiry date: " + cf_data["expiry"])
    print("Current Price: " + str(cf_data["current"]))
    print("Strike: " + str(cf_data["strike"]))
    print("Actual delta: " + str(cf_data["delta"] * 100) + "%")
    print("Start Date: " + cf_data["c_date"])
    print("Start Cost: " + str(cf_data["c_bid"]))
    print("Final Date: " + cf_data["f_date"])
    print("Final Bid: " + str(cf_data["f_bid"]))
    print("Final Ask: " + str(cf_data["f_ask"]))
    print("Final Mid: " + str(cf_data["f_mid"]))
    print("Simple Yield: " + str(cf_data["s_yield"] * 100) + "%")
    print("Annualized Yield: " + str(cf_data["a_yield"] * 100) + "%")
    
def confirm_bid_available(results,cf,start_date,start_date_stick):
    if cf["c_bid"] == 0.0:
        while True:
            start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
            start_date = start_date + datetime.timedelta(days=1)
            away = start_date - start_date_stick
            if away.days >= 10:
                print(" -> No available trade dates within 10 days")
                break
            start_date = start_date.strftime('%Y-%m-%d')
            cf = cost_final(ticker,start_date,None,delta,fwd,verbose)
            ed = datetime.datetime.strptime(cf['expiry'], "%Y-%m-%d")
            if cf["c_bid"] > 0.0:
                if (datetime.datetime.strptime(cf['trade_date'], "%Y-%m-%d") - start_date).days > 10:
                    print(" -> Pass, trade date more than 10 days away")
                    break
                else:
                    final_cf = cf
                    break
    else:
        final_cf = cf
    return final_cf

def write_csv(start_date,results):
    print(" --> Done. Writing CSV file")

    with open('option_results_' + start_date + '.csv', 'w') as csvfile:
        fieldnames = results.values()[0].keys()
        print(results)
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for t in results:
            ticker = t
            data = results[t]
            writer.writerow(data)

def write_total_csv(total_pl):
    print(" -----> Writing Final CSV file")

    with open('total_pl.csv', 'w') as csvfile:
        w = csv.writer(csvfile)
        w.writerow(total_pl[0].keys())
        for r in total_pl:
            w.writerow(r.values())

In [52]:
def testing():
    ticker = 'AAPL'
    start_date = '2007-01-01'
    end_date = None
    start_date_stick = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    delta = 0.9
    fwd = 40
    verbose = True

    cf = cost_final(ticker,start_date,end_date,delta,fwd,verbose)
    return cf
    #pretty_print(cf)
    if cf["c_bid"] == 0.0:
        print('.............................')
        print("No bid on this date, trying next date")
        print('.............................')
        while True:
            start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
            start_date = start_date + datetime.timedelta(days=1)
            away = start_date - start_date_stick
            print("Looking ahead " + str(away.days) + " day(s)")
            if away.days >= 10:
                print("No bids within 10 days of target")
                break
            start_date = start_date.strftime('%Y-%m-%d')
            cf = cost_final(ticker,start_date,end_date,delta,fwd,verbose)
            if cf["c_bid"] > 0.0:
                return cf
                #pretty_print(cf)
                break

testing()

Security ID: 101594.0
Adjusted Start Date: 2007-01-08
Available ex dates: 
[datetime.date(2008, 1, 19) datetime.date(2007, 1, 20)
 datetime.date(2007, 2, 17) datetime.date(2007, 4, 21)
 datetime.date(2007, 7, 21) datetime.date(2009, 1, 17)]
Target ex date: 2007-02-17
Nearest ex date: 2007-02-17
Available strikes: 
[125.0, 105.0, 110.0, 115.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100.0]
Current price: 85.91
Target strike: 77.32
Closest strike: 75.0
.............................
.............................


[{'a_yield': 0.1628,
  'c_ask': 1.3,
  'c_bid': 1.25,
  'c_date': '2007-01-08',
  'c_mid': 1.267,
  'current': 85.91,
  'delta': -0.13,
  'expiry': '2007-02-17',
  'f_ask': 0.05,
  'f_bid': 0.0,
  'f_date': '2007-02-17',
  'f_mid': 0.034,
  'gross_profit': 1.2329999999999999,
  's_yield': 0.0169,
  'strike': 75.0,
  'ticker': 'AAPL',
  'trade_date': '2007-01-08'}]

In [8]:
#db.list_tables('optionm')

In [5]:
#db.describe_table('compm', 'company')

In [30]:
def previous_quarter(date_string):
    ref = datetime.datetime.strptime(date_string, "%Y-%m-%d").date()
    if ref.month < 4:
        return datetime.date(ref.year - 1, 12, 31)
    elif ref.month < 7:
        return datetime.date(ref.year, 3, 31)
    elif ref.month < 10:
        return datetime.date(ref.year, 6, 30)
    return datetime.date(ref.year, 9, 30)

def top_n_mkt_cap_gvkeys(date_string,n,gvkeys):
    date1 = previous_quarter(date_string)
    date2 = previous_quarter(date1.strftime('%Y-%m-%d'))
    gvkeys_list = tuple(["'%s'"%str(i) for i in gvkeys])
    gvkeys_list = ",".join(gvkeys_list)
    q1 = ("select datadate, gvkey,cshoq,prccq,cshoq*prccq as market_cap, curcdq "
            "from compm.fundq "
           "where cshoq>0 and prccq>0 and curcdq='USD' and datadate > '%s' and datadate <= '%s' and gvkey in (%s) "
         "order by market_cap desc "
        "limit %i") % (date2,date1,gvkeys_list,n)
    df = db.raw_sql(q1)
    return list(df.gvkey.unique())

def gvkeys_for_filtered_security_type(date_string):
    year = date_string[0:4]
    date1 = previous_quarter(date_string)
    date2 = previous_quarter(date1.strftime('%Y-%m-%d'))
    q1 = ("select gvkey,cyear,tpci,exchg,datadate "
            "from compm.secm "
           "where datadate > '%s' and datadate <= '%s' and tpci in ('0','F') and exchg in (11,12,14,19) ")% (date2,date1)
    df = db.raw_sql(q1)
    return list(df.gvkey.unique())

def gvkeys_for_filtered_sectors(date_string,gvkeys):
    gvkeys_list = tuple(["'%s'"%str(i) for i in gvkeys])
    gvkeys_list = ",".join(gvkeys_list)
    q1 = ("select gvkey,gsector,loc "
            "from compm.company "
           "where gsector != '40' and gvkey in (%s) and loc='USA' ")% gvkeys_list
    df = db.raw_sql(q1)
    return list(df.gvkey.unique())

def get_latest_prices(date_string,gvkeys):
    gvkeys_list = tuple(["'%s'"%str(i) for i in gvkeys])
    gvkeys_list = ",".join(gvkeys_list)
    q1 = ("select datadate,gvkey,prccd,tpci "
            "from compm.secd "
           "where datadate = '%s' and gvkey in (%s) and tpci in ('0','F') ") % (date_string,gvkeys_list)
    df = db.raw_sql(q1)
    while True:
        if df.gvkey.nunique() == 0:
            date = datetime.datetime.strptime(date_string, "%Y-%m-%d") - datetime.timedelta(days=1)
            date_string = date.strftime('%Y-%m-%d')
            q1 = ("select datadate,gvkey,prccd,tpci "
            "from compm.secd "
           "where datadate = '%s' and gvkey in (%s) and tpci in ('0','F') ") % (date_string,gvkeys_list)
            df = db.raw_sql(q1)
        else:
            break
    return df

def get_cheapest_50_tics(date_string,gvkeys):
    date1 = previous_quarter(date_string)
    date2 = previous_quarter(date1.strftime('%Y-%m-%d'))
    date3 = date2 - relativedelta(years=1)
    gvkeys_list = tuple(["'%s'"%str(i) for i in gvkeys])
    gvkeys_list = ",".join(gvkeys_list)
    q1 = ("select datadate,gvkey,tic,cusip,oiadpq,cshoq,prccq,pstkq, dlcq, dlttq, cheq "
            "from compm.fundq "
           "where datadate <= '%s' and datadate > '%s' and gvkey in (%s) ") % (date2,date3,gvkeys_list)
    df = db.raw_sql(q1)
    df["date"] = pd.to_datetime(df['datadate'], format='%Y-%m-%d')
    df.sort_values(by=["date"])
    df = df.groupby('gvkey').ffill()
    df['oiadpq_ttm'] = df.groupby('gvkey')['oiadpq'].apply(lambda x: x.ffill().rolling(window=4).sum())
    latest_prices = get_latest_prices(date_string,gvkeys)
    df = pd.merge(df, latest_prices, on = 'gvkey', how='left')
    df["market_cap"] = df["cshoq"] * df["prccd"]
    df["ent_value"] = df["market_cap"] + df["pstkq"] + df["dlcq"] + df["dlttq"] - df["cheq"]
    df["ev_ebit"] = (df["ent_value"] / df["oiadpq_ttm"])
    df = df.round({'market_cap': 2, 'ent_value': 2, 'ev_ebit' : 2})
    df = df.loc[df['ev_ebit'] > 0]
    df = df.sort_values(by=["ev_ebit"])
    df = df.head(50)
    tics = [t.split('.')[0] for t in list(df.tic.unique())]
    
    #return df
    return tics

def get_tics(date_string):
    gvkeys = gvkeys_for_filtered_security_type(date_string)
    sector_filtered = gvkeys_for_filtered_sectors(date_string,gvkeys)
    top_n_gvkeys = top_n_mkt_cap_gvkeys(date_string,1000,sector_filtered)
    d = get_cheapest_50_tics(date_string,top_n_gvkeys)
    return d

In [ ]:
rebal_data = [['2007-01-01','2007-01-31',
        ['XOM','CVX','COP','OXY','ECA','MRO','VLO','AET','APA','TLM','NUE','TECK','PCAR','SCCO','EOG','FCX','X','NBL','LEN','1086832D','CMI','ANDV','PTEN','VRX','FTO','CMC','STLD','RS','PDS','MEOH','HFC','CRS','MDC','FRO','CECO','LPX','UNT','OSGIQ','YRCW','EXP','CNW','SM','CLF','IDCC','MTH','SCHN','OLN','MLI','GMRRQ','KSWS']
        ],
        ['2007-02-01','2007-02-28',
        ['XOM','CVX','COP','OXY','ECA','MRO','VLO','AET','APA','TLM','NUE','TECK','PCAR','SCCO','EOG','FCX','X','NBL','LEN','1086832D','CMI','ANDV','PTEN','VRX','FTO','CMC','STLD','RS','PDS','MEOH','HFC','CRS','MDC','FRO','CECO','LPX','UNT','OSGIQ','YRCW','EXP','CNW','SM','CLF','IDCC','MTH','SCHN','OLN','MLI','GMRRQ','KSWS']
        ]
       ]

In [43]:
delta = 0.9
fwd = 40
verbose = False
starting_capital = 1000000
max_size = 0.10

start_date = '2015-01-01'
stop_date = '2015-01-30'

total_pl = []
total_pl.append({'end_date' : start_date,
                'market_value' : starting_capital})

while True:
    if datetime.datetime.strptime(start_date, "%Y-%m-%d") > datetime.datetime.strptime(stop_date, "%Y-%m-%d"):
        break
    print("Getting tickers for " + start_date)
    tics = get_tics(start_date)
    if verbose:
        print(tics)
    results = {}
    count = 0.0
    print("Calculating ")
    for t in tics[0:3]:
        count = count + 1
        i = (count/len(tics))*100
        sys.stdout.write("\r%d%%" % i)
        sys.stdout.flush() 
        start_date_stick = datetime.datetime.strptime(start_date,"%Y-%m-%d")
        try:
            if verbose:
                print("Calculating " + str(t))
            cf = cost_final(t,start_date,None,delta,fwd,verbose)
            if (datetime.datetime.strptime(cf['trade_date'], "%Y-%m-%d") - start_date_stick).days > 10:
                if verbose:
                    print(" -> Pass, trade date more than 10 days away")
                continue
            results[t] = confirm_bid_available(results,cf,start_date,start_date_stick)
        except:
            if verbose:
                print(" -> Ticker " + str(t) + " wasn't able to retrieve data")
            pass
    print(results)
    results = calculate_number_of_contracts(starting_capital,results,max_size)

    starting_capital = update_starting_capital(results)
    print(results)
    write_csv(start_date,results)
    
    start_date = (average_ex_date(results) + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    
    total_pl.append(update_total_pl(start_date,total_pl))
    print(total_pl[-1]["market_value"])

write_total_csv(total_pl)

Getting tickers for 2015-01-01
Calculating 
6%[{'c_ask': 0.8, 'a_yield': 0.17, 'c_date': '2015-01-02', 'gross_profit': 0.632, 'f_date': '2015-02-20', 'delta': -0.11, 'c_mid': 0.733, 'expiry': '2015-02-20', 'current': 38.03, 's_yield': 0.0216, 'f_bid': 0.0, 'trade_date': '2015-01-02', 'strike': 34.0, 'f_mid': 0.101, 'c_bid': 0.7, 'f_ask': 0.15, 'ticker': u'WNR'}, {'c_ask': 0.83, 'a_yield': 0.1304, 'c_date': '2015-01-02', 'gross_profit': 0.602, 'f_date': '2015-02-13', 'delta': -0.1, 'c_mid': 0.649, 'expiry': '2015-02-13', 'current': 49.77, 's_yield': 0.0144, 'f_bid': 0.0, 'trade_date': '2015-01-02', 'strike': 45.0, 'f_mid': 0.047, 'c_bid': 0.56, 'f_ask': 0.07, 'ticker': u'VLO'}]


AttributeError: 'list' object has no attribute 'values'